In [3]:
import os
import re
import csv
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pathlib
from pathlib import Path
import pandas as pd

In [ ]:
BASE = "https://www.midiworld.com"
CSV_FILE = "movie_themes.csv"
OMDB_KEY = "f4d61a0f"        # ← free at omdbapi.com

# IMDb website

Collect Imdb ids from the list of movies in the source code collected from the webpage.

In [ ]:
html_page = "../data/metadata/imdb_top200_source.html"

In [6]:
html = pathlib.Path(html_page).read_text(encoding="utf-8")
soup = BeautifulSoup(html, "html.parser")

imdb_ids = []
pattern = r'https?://www\.imdb\.com/title/(tt\d+)'
movie_ids = re.findall(pattern, html)

For each id, collect relevant attributes for the respective movie.

In [7]:
 
movies = []

for id in movie_ids:
    url = f"http://www.omdbapi.com/?apikey={OMDB_KEY}&i={id}"
    r = requests.get(url)
    # soup = BeautifulSoup(response.text)


    if r.ok and r.json().get("Response") == "True":
        data = r.json()
        time.sleep(0.5)
        movies.append({
            "Title": data.get("Title"),
            "Released": data.get("Released"),
            "Director": data.get("Director"),
            "imdbRating": data.get("imdbRating"),
            "Ratings": data.get("Ratings"),
            "Genre": data.get("Genre"),
            "ID": id
        })

Save the collected data into a csv file

In [ ]:
with open("../data/metadata/movies_metadata.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["ID","Title","Released","Director","imdbRating", "Ratings","Genre"])
    writer.writeheader()
    for m in movies:

        row = {
            "ID": m["ID"],
            "Title": m["Title"],
            "Released": m["Released"],
            "Director": m["Director"],
            "imdbRating": m["imdbRating"],
            "Ratings": m["Ratings"],
            "Genre": m["Genre"]
        }
        writer.writerow(row)

In [ ]:
import pandas as pd
import os

# Load the CSV
movies_df = pd.read_csv("../data/metadata/movies_metadata.csv")

# List of downloaded MIDI files (assuming filenames are exactly the IMDb IDs)
midi_files = os.listdir("../data/midis")  # folder containing .mid files
midi_ids = [f.replace(".mid", "") for f in midi_files]

# First column in movies_metadata.csv contains IMDb IDs
movie_ids = movies_df.iloc[:, 0].astype(str).tolist()

# Find movies not downloaded yet
not_downloaded = [mid for mid in movie_ids if mid not in midi_ids]

print("Movies in CSV but not downloaded as MIDI:")
counter = 0
for movie in not_downloaded:
    counter += 1
print(counter)

Movies in CSV but not downloaded as MIDI:
113
